<a href="https://colab.research.google.com/github/mybenkhadda/docxchain/blob/main/docxchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/mybenkhadda/docxchain.git

Cloning into 'docxchain'...
remote: Enumerating objects: 403, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 403 (delta 6), reused 7 (delta 0), pack-reused 364
Receiving objects: 100% (403/403), 253.25 MiB | 14.04 MiB/s, done.
Resolving deltas: 100% (61/61), done.
Updating files: 100% (342/342), done.


In [1]:
%cd docxchain

/content/docxchain


In [2]:
! pip install -q pdf2image ipdb modelscope datasets==2.18.0 rapid_latex_ocr pyclipper

In [3]:
!pip install --upgrade "tensorflow<=2.10"

In [4]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.3 [186 kB]
Fetched 186 kB in 1s (133 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121752 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.3_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.3) ...
Setting up poppler-utils (22.02.0-2ubuntu0.3) ...
Processing triggers for man-db (2.10.2-1) ...


In [44]:
import sys
from pdf2image import convert_from_path
from PIL import Image
import argparse
import os
import numpy as np
import cv2
import datetime
import random
import time
import pytz
import json
import pandas as pd
import matplotlib.pyplot as plt
from pipelines.document_structurization import DocumentStructurization
from utilities.visualization import *

In [6]:
configs = dict()

layout_analysis_configs = dict()
layout_analysis_configs['from_modelscope_flag'] = False
layout_analysis_configs['model_path'] = './home/DocXLayout_231012.pth'  # note that: currently the layout analysis model is NOT from modelscope
configs['layout_analysis_configs'] = layout_analysis_configs

text_detection_configs = dict()
text_detection_configs['from_modelscope_flag'] = True
text_detection_configs['model_path'] = 'damo/cv_resnet18_ocr-detection-line-level_damo'
configs['text_detection_configs'] = text_detection_configs

text_recognition_configs = dict()
text_recognition_configs['from_modelscope_flag'] = True
text_recognition_configs['model_path'] = 'damo/cv_convnextTiny_ocr-recognition-document_damo'  # alternatives: 'damo/cv_convnextTiny_ocr-recognition-scene_damo', 'damo/cv_convnextTiny_ocr-recognition-general_damo', 'damo/cv_convnextTiny_ocr-recognition-handwritten_damo'
configs['text_recognition_configs'] = text_recognition_configs

formula_recognition_configs = dict()
formula_recognition_configs['from_modelscope_flag'] = False
formula_recognition_configs['image_resizer_path'] = '/home/LaTeX-OCR_image_resizer.onnx'
formula_recognition_configs['encoder_path'] = '/home/LaTeX-OCR_encoder.onnx'
formula_recognition_configs['decoder_path'] = '/home/LaTeX-OCR_decoder.onnx'
formula_recognition_configs['tokenizer_json'] = '/home/LaTeX-OCR_tokenizer.json'
configs['formula_recognition_configs'] = formula_recognition_configs

In [48]:
random.seed(1)

In [7]:
document_structurizer = DocumentStructurization(configs)

Fix size testing.
training chunk_sizes: [32]
The output will be saved to  /content/docxchain/../../exp/ctdet_subfield/default
heads {'hm': 11, 'cls': 4, 'ftype': 3, 'wh': 8, 'hm_sub': 2, 'wh_sub': 8, 'reg': 2, 'reg_sub': 2}
[0]
--> loading model from local file: ./home/DocXLayout_231012.pth


2024-04-25 07:37:38,645 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
Downloading: 100%|██████████| 312M/312M [00:13<00:00, 24.5MB/s]
Downloading: 100%|██████████| 41.7k/41.7k [00:00<00:00, 249kB/s]
Downloading: 100%|██████████| 8.06M/8.06M [00:00<00:00, 11.2MB/s]
Downloading: 0.00B [00:00, ?B/s]
Downloading: 100%|██████████| 118/118 [00:00<00:00, 183kB/s]
Downloading: 100%|██████████| 61.7k/61.7k [00:00<00:00, 339kB/s]
Downloading: 100%|██████████| 313k/313k [00:00<00:00, 960kB/s]
Downloading: 100%|██████████| 436k/436k [00:00<00:00, 1.04MB/s]
Downloading: 100%|██████████| 24.0/24.0 [00:00<00:00, 86.9kB/s]
Downloading: 100%|██████████| 3.93k/3.93k [00:00<00:00, 2.63MB/s]
2024-04-25 07:38:03,542 - modelscope - INFO - initiate model from /root/.cache/modelscope/hub/damo/cv_resnet18_ocr-detection-line-level_damo
2024-04-25 07:38:03,544 - modelscope - INFO - initiate model from location /root/.cache/modelscope/hub/damo/cv_resnet18_ocr-detection-line-level_dam

In [39]:
def pdf2image(pdf_path):
    """
    Convert a PDF file to an image.
    """
    images = convert_from_path(pdf_path)

    for i in range(len(images)):

        images[i] = np.array(images[i])

    return images

In [62]:
def extractFooter(image):
    image = image[:2200,:, :]
    fotter = image[2200:,:, :]
    return image, fotter

In [63]:
def merge_regions(results):
    regions = []
    for page in range(len(results) - 1):
        if results[page]["information"][-1]["category_name"] == results[page+1]["information"][0]["category_name"]:
            if results[page]["information"][-1]["category_name"] == "table":
                regions.append({
                    "page": page,
                    "region_poly1": results[page]["information"][-1]["region_poly"],
                    "region_poly2": results[page+1]["information"][0]["region_poly"],
                })
                results[page]["information"][-1] = {}
                results[page+1]["information"][0] = {}

    return results, regions

In [42]:
def order_file(results):
    results.sort(key=lambda x: (x["region_poly"][1]))
    return results

In [43]:
def structure1image(image):

    final_result = document_structurizer(image)

    final_result = order_file(final_result)

    document_structurization_visualization(final_result, image)

    return final_result

In [46]:
def document_structure(path):
    images = pdf2image(path)
    images = [extractFooter(image)[0] for image in images]

    doc = [
        {
            "page": 0,
            "content": structure1image(images[0])
        }
    ]

    if len(images) > 1:
        for i in range(1, len(images)):
            tmp = structure1image(images[i])
            if doc[i-1]["content"][-1]["category_name"] == tmp[0]["category_name"] and doc[i-1]["content"][-1]["category_name"] == "table":
                img1 = images[i-1][doc[i-1]["content"][-1]["region_poly"][1]:doc[i-1]["content"][-1]["region_poly"][-1],:,:]
                img2 = images[i][tmp[0]["region_poly"][1]:tmp[0]["region_poly"][-1],:,:]

                concatenated_img = np.concatenate((img1, img2), axis=0)

                plt.imsave(f"tables/table-{random.randint(0,100)}-page{i}.jpg", concatenated_img)

                # plt.imshow(concatenated_img)

                # result_merge = structure1image(concatenated_img)

                doc[i-1]["content"] = doc[i-1]["content"][:-2]



                doc.append(
                    {
                        "page": i,
                        "content": structure1image(images[i])[1:]
                    }
                )

            else:
                doc.append({
                    "page": i,
                    "content": structure1image(images[i])
                })

    return(doc)


In [ ]:
output = document_structure("purchasing_contract_example.pdf")

In [55]:
def json2df(jsonData):
  # try:
    data = []
    for i in range(len(jsonData)):
      page = jsonData[i]["content"]
      for j in range(len(page)):
        region_poly = page[j]["region_poly"]
        category_name = page[j]["category_name"]
        content = " ".join([page[j]["text_list"][k]["content"][0] for k in range(len(page[j]["text_list"]))])
        page_number = i
        data.append(
            {
                "region": region_poly,
                "category_name": category_name,
                "content": content,
                "page": int(page_number)
            }
        )
    return pd.DataFrame(data, columns = data[0].keys())
  # except:
  #   print(page[j])

In [56]:
df = json2df(output)

In [60]:
def extractTables(pdf, df):
  images = pdf2image(pdf)
  tables = df[df["category_name"] == "table"]
  for i in range(tables.shape[0]):
    region = tables.iloc[0]["region"]
    page = tables.iloc[0]["page"]
    img = images[page][region[1]:region[3], region[2]:region[6],:]
    plt.imsave(f"tables/table-{random.randint(0,100)}-page{tables.iloc[0]['page']}.jpg", img)


In [61]:
extractTables("purchasing_contract_example.pdf", df)

9


ValueError: cannot write empty image as JPEG